In [1]:
import psycopg2
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Connect to Redshift
conn = psycopg2.connect(
    dbname=os.getenv('REDSHIFT_SCHEMA'),
    user=os.getenv('REDSHIFT_USER'),
    password=os.getenv('REDSHIFT_PASSWORD'),
    host=os.getenv('REDSHIFT_HOST'),
    port=5439
)

cur = conn.cursor()

# Example schema — adjust columns to match your CSV
create_table_sql = """
DROP TABLE IF EXISTS DAILY_CRIPTO_PRICES;

CREATE TABLE DAILY_CRIPTO_PRICES (
    usd FLOAT,
    cripto VARCHAR(100),
    date TIMESTAMP
);
"""

cur.execute(create_table_sql)
conn.commit()

#cur.close()

In [3]:
df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)
print(df.head())

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_1796/2560394726.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)


Empty DataFrame
Columns: [usd, cripto, date]
Index: []


In [4]:
prices_historic=pd.DataFrame()
for currency in ["bitcoin","ethereum","tether","solana"]:
    currency_historic = pd.read_csv(f'~/Downloads/{currency}-usd-max.csv',
    dtype={"total_volume": "float64", "price": "float64","market_cap":"float64"},
    parse_dates=["snapped_at"],
    date_parser=lambda x: pd.to_datetime(x, utc=True).strftime("%d-%m-%Y"))
    currency_historic.columns = ['date', 'usd', 'market_cap', 'total_volume']
    currency_historic = currency_historic[['date', 'usd']]
    currency_historic['cripto'] = currency
    prices_historic = pd.concat([prices_historic,currency_historic],ignore_index=True)

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_1796/3158066530.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  currency_historic = pd.read_csv(f'~/Downloads/{currency}-usd-max.csv',
/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_1796/3158066530.py:3: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  currency_historic = pd.read_csv(f'~/Downloads/{currency}-usd-max.csv',
/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_1796/3158066530.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  currency_historic = pd.read_csv(f'~/Downloads/{cur

In [5]:
prices_historic['date'] = prices_historic['date'].astype('datetime64[ns]')

In [6]:
prices_historic[prices_historic['date']>='2023-01-01']

,date,usd,cripto
3533,2023-01-01,16540.693625,bitcoin
3534,2023-01-02,16615.969318,bitcoin
3535,2023-01-03,16674.342536,bitcoin
3536,2023-01-04,16677.646186,bitcoin
3537,2023-01-05,16855.174739,bitcoin
...,...,...,...
14262,2025-09-11,158.156189,solana
14263,2025-10-11,164.268421,solana
14264,2025-11-11,167.548409,solana
14265,2025-12-11,154.346288,solana


In [7]:
from sqlalchemy import create_engine, text

engine = create_engine(os.getenv("REDSHIFT"))

# Insert a single row

for index, row in prices_historic[prices_historic['date']>='2023-01-01'].iterrows():
    with engine.connect() as conn:
        conn.execute(
            text("INSERT INTO DAILY_CRIPTO_PRICES (usd, cripto,date) VALUES (:usd, :cripto,:date)"),
            {"usd": row.usd, "cripto":row.cripto,'date':row.date}
            )
        conn.close()
    print(index)

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_1796/3744172360.py:9: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  conn.execute(


3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726
3727
3728
3729
3730
3731
3732


In [9]:
conn = psycopg2.connect(
    dbname=os.getenv('REDSHIFT_SCHEMA'),
    user=os.getenv('REDSHIFT_USER'),
    password=os.getenv('REDSHIFT_PASSWORD'),
    host=os.getenv('REDSHIFT_HOST'),
    port=5439
)
df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)
df

/var/folders/1k/sh8hpzz512gfjrxqfmzwshbc0000gn/T/ipykernel_1796/2335844845.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql("SELECT * FROM DAILY_CRIPTO_PRICES;", conn)


,usd,cripto,date
0,16540.693625,bitcoin,2023-01-01
1,16615.969318,bitcoin,2023-01-02
2,16674.342536,bitcoin,2023-01-03
3,16677.646186,bitcoin,2023-01-04
4,16855.174739,bitcoin,2023-01-05
...,...,...,...
4203,142.050000,solana,2025-10-11
4204,104709.678617,bitcoin,2025-10-11
4205,3576.254705,ethereum,2025-10-11
4206,0.999748,tether,2025-10-11
